In [1]:

import sys
sys.path.append("../")

import os


from desci_sense.twitter import scrape_tweet, extract_external_ref_urls
from desci_sense.utils import extract_and_expand_urls, extract_urls

from desci_sense.postprocessing.output_parsers import TagTypeParser

In [8]:
TEST_TWEET_W_REF = "https://twitter.com/victorveitch/status/1722300572554969090"
TEST_TWEET_W_REF_2 = "https://twitter.com/maksym_andr/status/1722235666724192688"
TEST_TWEET_WO_REF_1 = "https://twitter.com/mpshanahan/status/1722283975450722407"
test_url_w_img = "https://twitter.com/HarvardPSC/status/1722102271792603452"
TEST_TWEET_REF_TWITTER_EXT = "https://twitter.com/cognazor/status/1722598121887117753"

In [6]:
TEST_CASES = [

    ("https://twitter.com/victorveitch/status/1722300572554969090", True),
    ("https://twitter.com/maksym_andr/status/1722235666724192688", True),
    ("https://twitter.com/mpshanahan/status/1722283975450722407", False),
    ("https://twitter.com/victorveitch/status/1722303746397409698", False),
    ("https://twitter.com/HarvardPSC/status/1722102271792603452", False)
]

In [11]:
tweet = scrape_tweet("https://twitter.com/PetarV_93/status/1722528511343091943")
tweet

{'conversationID': '1722528511343091943',
 'date': 'Thu Nov 09 08:16:14 +0000 2023',
 'date_epoch': 1699517774,
 'hashtags': [],
 'likes': 77,
 'mediaURLs': [],
 'media_extended': [],
 'possibly_sensitive': False,
 'qrtURL': 'https://twitter.com/i/status/1722431083357036767',
 'replies': 1,
 'retweets': 7,
 'text': 'We used old-school (Tabu search) and new-school (AlphaZero) AI to discover new constructions in graph theory!\n\nTo appear at the #NeurIPS2023 MATH-AI Workshop :)\n\nhttps://arxiv.org/abs/2311.03583',
 'tweetID': '1722528511343091943',
 'tweetURL': 'https://twitter.com/PetarV_93/status/1722528511343091943',
 'user_name': 'Petar Veličković',
 'user_screen_name': 'PetarV_93'}

In [7]:
for url, has_url in TEST_CASES:
    tweet = scrape_tweet(url)
    print("all urls: ", extract_urls(tweet["text"]))
    print("External urls: ", extract_external_ref_urls(tweet))
    print("-------")

all urls:  ['https://t.co/8WXQVI4WBb']
External urls:  []
-------
all urls:  ['https://arxiv.org/abs/2311.01906', 'https://t.co/PlPRQVL9yc']
External urls:  ['https://arxiv.org/abs/2311.01906']
-------
all urls:  []
External urls:  []
-------
all urls:  []
External urls:  []
-------
all urls:  ['https://t.co/s78lv2IVjJ']
External urls:  []
-------


In [9]:
test_url_w_img = "https://twitter.com/HarvardPSC/status/1722102271792603452"
tweet = scrape_tweet(test_url_w_img)
tweet

{'conversationID': '1722102271792603452',
 'date': 'Wed Nov 08 04:02:31 +0000 2023',
 'date_epoch': 1699416151,
 'hashtags': ['CeasefireNow'],
 'likes': 4729,
 'mediaURLs': ['https://video.twimg.com/amplify_video/1722102054536273920/vid/avc1/1280x720/mOOsZBrIq0_5nZUf.mp4?tag=14'],
 'media_extended': [{'altText': None,
   'duration_millis': 206424,
   'size': {'height': 1080, 'width': 1920},
   'thumbnail_url': 'https://pbs.twimg.com/amplify_video_thumb/1722102054536273920/img/OtgNOUxiJyCx4J-V.jpg',
   'type': 'video',
   'url': 'https://video.twimg.com/amplify_video/1722102054536273920/vid/avc1/1280x720/mOOsZBrIq0_5nZUf.mp4?tag=14'}],
 'possibly_sensitive': False,
 'qrtURL': None,
 'replies': 453,
 'retweets': 2053,
 'text': '🧵 THREAD: Harvard Rabbi Zarchi preaches anti-Palestinian racism. In a speech to the community on Nov 4, he made a distinction between humans and “four-legged animals.” \n\nApparently, calling for a #CeasefireNow and an end to the occupation makes us “below animals

In [10]:
print(tweet["text"])

🧵 THREAD: Harvard Rabbi Zarchi preaches anti-Palestinian racism. In a speech to the community on Nov 4, he made a distinction between humans and “four-legged animals.” 

Apparently, calling for a #CeasefireNow and an end to the occupation makes us “below animals,” even… https://t.co/s78lv2IVjJ


In [13]:
print(extract_and_expand_urls(tweet["text"]))

['https://twitter.com/HarvardPSC/status/1722102271792603452/video/1']


In [ ]:
def extract_external_ref_urls(tweet):
    """
    Extract list of non-internal URLs referenced by this tweet (in the tweet text body).
    In this context, internal URLs are URLs of media items associated with the tweet, such as images or videos.
    Internal URLs share the same ID as the referencing tweet.
    Shortened URLs are expanded to long form.
    """
    urls = extract_and_expand_urls(tweet["text"])

In [15]:
def extract_status_id(url):
    pattern = r'twitter\.com\/\w+\/status\/(\d+)'
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    else:
        return None
    
# Example usage:
twitter_url = "https://twitter.com/HarvardPSC/status/1722102271792603452/video/1"
status_id = extract_status_id(test_url_w_img)
print(status_id)

1722102271792603452


In [16]:

urls = extract_and_expand_urls(tweet["text"])

external = []
for url in urls:
    twitter_id = extract_status_id(url)
    if twitter_id: # check if a twitter url
        if twitter_id != tweet["tweetID"]: # check if url shares same status id with parsed tweet
            external.append(url)

external

[]